In [1]:
import os
os.chdir('../')
!pwd

/home/lok/Documents/ML_Projects/rice_leaf_disease


In [4]:
from dataclasses import dataclass
from pathlib import Path
from rice_leaf_disease.constants import CONFIG_FILE_PATH, PARAMS_FILE_PATH
from rice_leaf_disease.utils.common import create_directories, read_yaml


In [5]:
@dataclass(frozen=True)
class EvaluationConfig:
    model_path: Path
    training_data: Path
    all_params: dict
    params_image_size: list
    

In [6]:
class ConfigurationManager:
    def __init__(
        self, 
        config_filepath = CONFIG_FILE_PATH,
        params_filepath = PARAMS_FILE_PATH):
        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)
        create_directories([self.config.artifacts_root])
    
    def get_evaluation_config(self) -> EvaluationConfig:
        eval_config = EvaluationConfig(
            model_path=Path('artifacts/training/model.keras'),
            training_data=Path('artifacts/data_ingestion/rice_leaf_diseases'),
            all_params=self.params,
            params_image_size=self.params.model_params.input_shape
        )
        
        return eval_config


In [ ]:
import tensorflow as tf

2024-12-29 17:35:22.568171: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-12-29 17:35:25.780972: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


[2024-12-29 17:35:27,929: INFO : utils : NumExpr defaulting to 8 threads.]


In [9]:
import numpy as np
import cv2
from PIL import Image

In [10]:
import json


class Evaluation:
    def __init__(self, config: EvaluationConfig):
        self.config = config
    
    @staticmethod
    def load_model(path: Path) -> tf.keras.Model:
        if not path.exists():
            raise FileNotFoundError(f"Model file not found at {path}")
        return tf.keras.models.load_model(path)
    
    @staticmethod
    def preprocess_data(data_path: Path, image_size: tuple) -> tuple:
        """
        Load and preprocess the dataset for evaluation.
        """
        bacteria = list(data_path.glob("Bacterial leaf blight/*"))
        brown = list(data_path.glob("Brown spot/*"))
        smut = list(data_path.glob("Leaf smut/*"))
        data = {"bacteria": bacteria, "brown": brown, "smut": smut}
        
        labels_dict = {
            'bacteria': 0,
            'brown': 1,
            'smut': 2
        }
        X, y = [], []

        for class_name, images in data.items():
            for image in images:
                img = cv2.imread(str(image))
                if img is None:
                    print(f"Failed to load image: {image}")
                    continue
                resized_img = cv2.resize(img, image_size[:2])
                X.append(resized_img)
                y.append(labels_dict[class_name])
        
        X = np.array(X) / 255.0  # Normalize images
        y = np.array(y)  # Labels
        return X, y
    
    def evaluation(self) -> dict:
        """
        Evaluate the model's performance on the dataset.
        """
        model = self.load_model(self.config.model_path)
        
        image_size = tuple(self.config.params_image_size)
        X, y = self.preprocess_data(self.config.training_data, image_size)

        print("Evaluating the model...")
        scores = model.evaluate(X, y, verbose=1)
        
        # Create a dictionary with metric names and values
        metrics = dict(zip(model.metrics_names, scores))
        print(f"Evaluation Metrics: {metrics}")
        return metrics
    
    def save_score(self, scores: dict, output_path: Path = Path('artifacts/evaluation/score.json')):
        """
        Save the evaluation metrics to a JSON file.
        """
        output_path.parent.mkdir(parents=True, exist_ok=True)
        with open(output_path, 'w') as f:
            json.dump(scores, f, indent=4)
        print(f"Scores saved to {output_path}")

In [ ]:
try:
    config_manager = ConfigurationManager()
    eval_config = config_manager.get_evaluation_config()
    evaluation = Evaluation(config=eval_config)
    
    scores = evaluation.evaluation()
    
    evaluation.save_score(scores)

except Exception as e:
    print(f"Error during evaluation: {e}")
    raise e


[2024-12-29 18:06:12,091: INFO : common : YAML file: config/config.yaml loaded successfully!]
[2024-12-29 18:06:12,106: INFO : common : YAML file: params.yaml loaded successfully!]
[2024-12-29 18:06:12,109: INFO : common : created directory at: artifacts]
Evaluating the model...
4/4 ━━━━━━━━━━━━━━━━━━━━ 2s 352ms/step - accuracy: 0.7925 - loss: 0.5781
Evaluation Metrics: {'loss': 0.563230037689209, 'compile_metrics': 0.7833333611488342}
Scores saved to artifacts/evaluation/score.json
